In [1]:
# Alle benötigten libraries
import pandas as pd
import numpy as np
import os

In [2]:
os.getcwd()

'c:\\celver\\scm_optimization\\v_b\\code'

# Bestellübersicht -Daten ab 2019 

In [3]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
bestell_df=pd.read_excel(excel,'Bestellübersicht')
bestell_df.head()

,Name des Lieferanten,Abgehendes DC intern,Werk,Einkaufsbeleg,intern/extern,Belegart,Position,Belegdatum,Material,Kurztext,BestellpreisME,Bestellmenge,noch zu liefern (Menge)
0,99011 Villeroy & Boch AG DC,11.0,47,49384564,intern,UBA,1,2024-01-31,5614R001P,Subway 2.0 WWC DirectFlush verpackt,ST,24,0
1,99099 Villeroy & Boch KFT DC,99.0,57,49384528,intern,UBA,3,2024-01-30,92219900,Fixation Set WWC Suprafix 3.0,ST,5000,0
2,90315 PRESSALIT A/S,NaN,11,57783718,extern,NB,1,2024-01-30,9M78S101,Subway 2.0 WC Sitz softcl/quick-rel.,ST,900,0
3,99011 Villeroy & Boch AG DC,11.0,42,49384482,intern,UBA,18,2024-01-29,560050R1P,Subway 2.0 WWC for ViClean packed,ST,12,12
4,99078 UCOSAN B.V.,78.0,58,49384488,intern,UBA,1,2024-01-29,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,ST,8,0


In [11]:
bestell_df[['Lieferantennummer','Lieferantenname']]=bestell_df['Name des Lieferanten'].str.split(' ',n=1,expand=True)

bestell_df.drop(['Name des Lieferanten'],axis=1,inplace=True)
first_column = bestell_df.pop('Lieferantennummer')
second_column = bestell_df.pop('Lieferantenname')  

#Reihenfolge angepasst
bestell_df.insert(0, 'Lieferantennummer', first_column)
bestell_df.insert(1, 'Lieferantenname', second_column)
bestell_df['Lieferantenname']=bestell_df['Lieferantenname'].apply(str.lstrip)
# Ergebnis anzeigen
bestell_df.head(100)

,Lieferantennummer,Lieferantenname,Abgehendes DC intern,Werk,Einkaufsbeleg,intern/extern,Belegart,Position,Belegdatum,Material,Kurztext,BestellpreisME,Bestellmenge,noch zu liefern (Menge)
0,99011,Villeroy & Boch AG DC,11.0,47,49384564,intern,UBA,1,2024-01-31,5614R001P,Subway 2.0 WWC DirectFlush verpackt,ST,24,0
1,99099,Villeroy & Boch KFT DC,99.0,57,49384528,intern,UBA,3,2024-01-30,92219900,Fixation Set WWC Suprafix 3.0,ST,5000,0
2,90315,PRESSALIT A/S,NaN,11,57783718,extern,NB,1,2024-01-30,9M78S101,Subway 2.0 WC Sitz softcl/quick-rel.,ST,900,0
3,99011,Villeroy & Boch AG DC,11.0,42,49384482,intern,UBA,18,2024-01-29,560050R1P,Subway 2.0 WWC for ViClean packed,ST,12,12
4,99078,UCOSAN B.V.,78.0,58,49384488,intern,UBA,1,2024-01-29,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,ST,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,99011,Villeroy & Boch AG DC,11.0,47,49382138,intern,UBA,10,2023-11-22,92249068,ViConnect 200S Betätigungs Pl. dual fl,ST,28,0
96,99028,Villeroy & Boch Trading,28.0,11,49382160,intern,UBA,25,2023-11-22,TVS00035100000,Univ. Vi-Box,ST,576,0
97,158995,Guangdong Huayi Plumbing,NaN,28,57772454,extern,NB,1,2023-11-22,TVS00035100000,Univ. Vi-Box,ST,1200,0
98,158995,Guangdong Huayi Plumbing,NaN,28,57772454,extern,NB,2,2023-11-22,TVS00035100000,Univ. Vi-Box,ST,1200,0


In [14]:
bestell_df.head(100)
bestell_df['Lieferantenname'][2]

'PRESSALIT A/S'

## AFKO Tabelle

In [4]:
afko_df=pd.read_excel(excel,'AFKO neu')
afko_df.head()

,Auftrag,Eckendtermin,Eckstarttermin,Material,Iststarttermin,Istendtermin,Gesamte Auftragsmenge
0,1500909609,2019-06-24,2019-06-20,FP603476,NaT,NaT,18340
1,1501124011,2021-02-01,2021-01-21,FP603476,2021-01-21,2021-01-22,124250
2,1501124012,2021-02-01,2021-01-22,FP603476,2021-01-27,2021-02-03,120000
3,1501124522,2021-03-25,2021-03-02,FP603476,NaT,NaT,303086
4,1501124563,2021-03-01,2021-02-12,FP603476,2021-02-12,2021-02-15,175100


In [ ]:
df['Beleg_Anlege_Dat'] = df['Angel.am']
df.drop(['Angel.am', 'BelegDat'], axis=1, inplace=True)


In [8]:
def readData(path_excel: str) -> list:
    xls = pd.ExcelFile(path_excel)
    list = xls.sheet_names
    df_dic = dict()
    for sheet in list:
        df = pd.read_excel(xls, sheet)
        df_dic[sheet]=df
    return df_dic

data=readData(excel)

In [10]:
print(data.keys())

dict_keys(['Infos', 'Bestellübersicht', 'BestÜ neu', 'EKKO', 'EKKO neu', 'EKPO', 'EKPO neu', 'EKBE', 'EKBE neu', 'AFKO', 'AFKO neu', 'AUFK', 'AUFK neu', 'MVER', 'Absatz', 'IO', 'Gross Demand'])


# MVER

In [23]:
def mver_df(xls: pd.ExcelFile) -> pd.DataFrame:
    df_1 = pd.read_excel(xls, 'MVER', nrows=74)
    df_2 = pd.read_excel(xls, 'MVER', skiprows=76, nrows=160-76)
    
    def change_month_to_monthYear(df: pd.DataFrame) -> pd.DataFrame:
        """
        Change the name of the columns to the format MonthYear
        """
        val = str(df.columns[0])[2:]
        for title in df.columns[1:]:
            df.rename(columns={title: title + '.' + val}, inplace=True)
        return df

    def change_first_three_columns(df: pd.DataFrame) -> pd.DataFrame:
        """
        Change the name of the first three columns to the values of the first row
        """
        for i in range(3):
            df.rename(columns={df.columns[i]: df.iloc[0, i]}, inplace=True)
        return df

    df_2 = change_month_to_monthYear(df_2)
    df_1 = change_month_to_monthYear(df_1)

    df_2 = change_first_three_columns(df_2)
    df_1 = change_first_three_columns(df_1)
    df_2 = df_2.iloc[1:]
    df_1 = df_1.iloc[1:]

    df_1.drop(['Jahr'], axis=1, inplace=True)
    df_2.drop(['Jahr'], axis=1, inplace=True)
    df = df_1.merge(df_2, on=['Material', 'Werk'], how='outer')
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')
    return df
    
data=mver_df(excel)

In [26]:
data.tail()

,Material,Werk,Month,Absatz
2227,UBQ170OBE2V-01,0042,Dez.22,33
2228,UBQ170OBE2V-01,0058,Dez.22,3
2229,UBQ170OBE2V-01,0072,Dez.22,296
2230,UBQ170OBE2V-01,0078,Dez.22,361
2231,UBQ170OBE2V-01,0099,Dez.22,0


In [37]:
df_1 = pd.read_excel(excel, 'MVER', skiprows=76, nrows=160-76)
df_1.tail()

,2022,Unnamed: 1,Unnamed: 2,Jan,Feb,Mrz,Apr,Mai,Jun,Jul,Aug,Sep,Okt,Nov,Dez
78,UBQ170OBE2V-01,0042,2022,257,182,199,45,99,167,31,15,268,33,12,33
79,UBQ170OBE2V-01,0058,2022,2,41,52,32,58,51,2,0,12,24,16,3
80,UBQ170OBE2V-01,0072,2022,968,1360,788,664,448,671,481,0,698,247,654,296
81,UBQ170OBE2V-01,0078,2022,489,440,480,491,506,468,325,382,418,618,526,361
82,UBQ170OBE2V-01,0099,2022,0,0,0,0,1,0,0,0,0,0,0,0


In [32]:
val = str(df_1.columns[0])[2:]
for title in df_1.columns[1:]:
    df_1.rename(columns={title: title + '.' + val}, inplace=True)


df_1.head()


,2023,Unnamed: 1.23,Unnamed: 2.23,Jan.23,Feb.23,Mrz.23,Apr.23,Mai.23,Jun.23,Jul.23,Aug.23,Sep.23,Okt.23,Nov.23,Dez.23
0,Material,Werk,Jahr,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.,Kor.GesVerbr.
1,560050R1P,0010,2023,0,2,0,276,1014,298,294,0,1020,937,0,1020
2,560050R1P,0011,2023,435,1332,928,96,540,231,168,924,0,774,222,891
3,560050R1P,0013,2023,1650,1852,182,660,886,698,879,108,568,289,343,583
4,560050R1P,0028,2023,0,0,0,0,0,0,0,31,102,18,2,4


# Der folgende Code ist falsch, durch den Merge kriegt man eine falsche Zeilenanzahl. In der Datei Preprocessing wurde das verbessert


In [68]:
df_1 = pd.read_excel(excel, 'MVER', nrows=74)
df_2 = pd.read_excel(excel, 'MVER', skiprows=76, nrows=160-76)
    
def change_month_to_monthYear(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the columns to the format MonthYear
    """
    val = str(df.columns[0])[2:]
    for title in df.columns[1:]:
        df.rename(columns={title: title + '.' + val}, inplace=True)
    return df

def change_first_three_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the first three columns to the values of the first row
    """
    for i in range(3):
        df.rename(columns={df.columns[i]: df.iloc[0, i]}, inplace=True)
    return df

df_2 = change_month_to_monthYear(df_2)
df_1 = change_month_to_monthYear(df_1)

df_2 = change_first_three_columns(df_2)
df_1 = change_first_three_columns(df_1)
df_2 = df_2.iloc[1:]
df_1 = df_1.iloc[1:]

df_1.drop(['Jahr'], axis=1, inplace=True)
df_2.drop(['Jahr'], axis=1, inplace=True)
df = df_1.merge(df_2, on=['Material', 'Werk'], how='right')
df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')
df.tail()

,Material,Werk,Month,Absatz
1963,UBQ170OBE2V-01,0042,Dez.22,33
1964,UBQ170OBE2V-01,0058,Dez.22,3
1965,UBQ170OBE2V-01,0072,Dez.22,296
1966,UBQ170OBE2V-01,0078,Dez.22,361
1967,UBQ170OBE2V-01,0099,Dez.22,0


In [62]:
df_1 = pd.read_excel(excel, 'MVER', nrows=74)
df_2 = pd.read_excel(excel, 'MVER', skiprows=76, nrows=160-76)
    
def change_month_to_monthYear(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the columns to the format MonthYear
    """
    val = str(df.columns[0])[2:]
    for title in df.columns[1:]:
        df.rename(columns={title: title + '.' + val}, inplace=True)
    return df

def change_first_three_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Change the name of the first three columns to the values of the first row
    """
    for i in range(3):
        df.rename(columns={df.columns[i]: df.iloc[0, i]}, inplace=True)
    return df

df_2 = change_month_to_monthYear(df_2)
df_1 = change_month_to_monthYear(df_1)

df_2 = change_first_three_columns(df_2)
df_1 = change_first_three_columns(df_1)
df_2 = df_2.iloc[1:]
df_1 = df_1.iloc[1:]

df_1.drop(['Jahr'], axis=1, inplace=True)
df_2.drop(['Jahr'],axis=1,inplace=True)
df = pd.melt(df_1, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')
df_new = pd.melt(df_2, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')

df_final= pd.concat([df,df_new], axis=0, ignore_index=True)
df_final.tail()

,Material,Werk,Month,Absatz
1843,UBQ170OBE2V-01,0042,Dez.22,33
1844,UBQ170OBE2V-01,0058,Dez.22,3
1845,UBQ170OBE2V-01,0072,Dez.22,296
1846,UBQ170OBE2V-01,0078,Dez.22,361
1847,UBQ170OBE2V-01,0099,Dez.22,0


## Absatz Tabelle

In [121]:
def absatz_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.iloc[1:]
    df.drop(['Unnamed: 1', 'Geschäftsj./Periode', 'Gesamtergebnis'], axis=1, inplace=True)
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Artikel', 'Werk'], var_name='Month', value_name='Absatz') 
    df['Werk']=df['Werk'].apply(int) 
    
    return df

data= absatz_df(pd.read_excel(excel, 'Absatz'))
data.tail()

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_5416\356454184.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 1', 'Geschäftsj./Periode', 'Gesamtergebnis'], axis=1, inplace=True)
C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_5416\356454184.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


,Artikel,Werk,Month,Absatz
1920,UBQ170OBE2V-01,42,JAN 2024,4.0
1921,UBQ170OBE2V-01,58,JAN 2024,9.0
1922,UBQ170OBE2V-01,78,JAN 2024,424.0
1923,UBQ170OBE2V-01,99,JAN 2024,0.0
1924,FG0603809,54,JAN 2024,2180.0


In [125]:
df=pd.read_excel(excel, 'Absatz')
df = df.iloc[1:]
df.drop(['Gesamtergebnis'], axis=1, inplace=True)
df.rename({'Unnamed: 1':'Kurztext','Geschäftsj./Periode':'Gesellschaft'},axis=1, inplace=True)
df=df.melt(id_vars=df.columns[:4],var_name='Month', value_name='Absatz')
df['Werk']=df['Werk'].apply(int) 
df['Absatz']=data['Absatz'].fillna(0)
df.head()

,Artikel,Kurztext,Werk,Gesellschaft,Month,Absatz
0,560050R1P,Subway 2.0 WWC for V,11,VILLEROY & BOCH AG,JAN 2022,0.0
1,560050R1P,Subway 2.0 WWC for V,28,Villeroy&Boch Tradin,JAN 2022,0.0
2,560050R1P,Subway 2.0 WWC for V,42,Villeroy&Boch Tradin,JAN 2022,490.0
3,61630001,Loop & Friends Unter,11,VILLEROY & BOCH AG,JAN 2022,579.0
4,61630001,Loop & Friends Unter,28,Villeroy&Boch Tradin,JAN 2022,0.0


## neue Absatz


In [15]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Absatz 012019 - 022024.xlsx')

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [45]:
df=pd.read_excel(excel, "YV000024AV1NT1 - 2024-03-13T161")
df=df.iloc[12:]
df.reset_index(drop=True, inplace=True)
df.columns=df.iloc[0]
df.drop(0,inplace=True)
cols=df.columns.tolist()
cols[0]=df.iloc[0,0]
cols[1]='Kurztext'
cols[2]=df.iloc[0,2]
cols[3]='Gesellschaft'
df.columns=cols
df=df.iloc[2:]
df=df.melt(id_vars=df.columns[0:4],var_name='Month', value_name='Absatz')
df['Absatz']=df['Absatz'].fillna(0)


C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_9084\3963015249.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Absatz']=df['Absatz'].fillna(0)


In [46]:

df.tail()

,Artikel,Kurztext,Werk,Gesellschaft,Month,Absatz
17665,UBQ170OBE2V-01,Oberon Badewanne 170,0071,Villeroy & Boch Well,FEB 2024,0.00
17666,UBQ170OBE2V-01,Oberon Badewanne 170,0078,Villeroy & Boch Well,FEB 2024,80493.44
17667,UBQ170OBE2V-01,Oberon Badewanne 170,0099,Villeroy & Boch Magy,FEB 2024,0.00
17668,FG0603809,Vatette 1219 Blankin,0007,VILLEROY & BOCH AG,FEB 2024,0.00
17669,FG0603809,Vatette 1219 Blankin,0054,Villeroy & Boch Gust,FEB 2024,1004.84


## IO Tabelle

In [6]:
def io_df(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={'Year': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 25'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='value')
    df['Material'] = df['Material'].str.split().str[1]
    df['Werk'] = df['Werk'].str.split().str[0]
    df['Month'] = df['Month'].str[-6:]
    return df


daten=pd.read_excel(excel,'IO',header=1)
daten.rename(columns={'Year': 'Werk'}, inplace=True)
daten.drop(['Unnamed: 25'], axis=1, inplace=True)
daten=daten.iloc[:-2]
d=list()
chrr=list(daten['Werk'])
a=''
for i in range(0,len(chrr)):
    if chrr[i][0]=='-':
        a=chrr[i]
    d.append(a)

daten['neue Spalte']=pd.Series(d)
cols=daten.columns.tolist()
cols=cols[-1:]+cols[:-1]
daten=daten[cols]
for i in range(0,len(chrr)):
    if chrr[i][0]=='-':
        daten.drop(i,inplace=True)

daten['Werk']=daten['Werk'].apply(str.strip)
daten=daten[~daten['Werk'].str.startswith('Total')]


daten['neue Spalte']=daten['neue Spalte'].apply(lambda x: x[2:])
daten['neue Spalte']=daten['neue Spalte'].apply(str.strip)
material=list()
kurztext=list()
lol=daten['neue Spalte']
for wort in lol:
    index= wort.index(" ")
    material.append(wort[0:index])
    kurztext.append(wort[index:])

daten['material']=material
daten['kurztext']=kurztext


daten.reset_index(drop=True, inplace=True)
cols=daten.columns.tolist()
cols=cols[-2:]+cols[0:-2]
daten=daten[cols]
daten=daten.melt(id_vars=daten.columns[0:4],var_name='Month', value_name='Absatz')
daten['Month'] = daten['Month'].str[-6:]



daten['Absatz']=daten['Absatz'].fillna(0)

# Anzeigen der ersten Zeilen des optimierten DataFrames
print(daten['Absatz'].sum())


327812.0


### optimierte Code


In [12]:
# Lesen des 'IO'-Blatts der Excel-Datei mit einem Header in der ersten Zeile
daten = pd.read_excel(excel, 'IO', header=1)

# Umbenennen der 'Year'-Spalte zu 'Werk' und Entfernen einer unnötigen Spalte
daten.rename(columns={'Year': 'Werk'}, inplace=True)
daten.drop(columns=['Unnamed: 25'], inplace=True)

# Entfernen der letzten zwei Zeilen
daten = daten.iloc[:-2]

# Entfernen von führenden Leerzeichen in der 'Werk'-Spalte und Filtern von Zeilen, die mit "Total" beginnen
daten['Werk'] = daten['Werk'].str.strip()
daten = daten[~daten['Werk'].str.startswith('Total')]

# Erstellen einer neuen Spalte, die Werte aus 'Werk' übernimmt, wenn sie mit '-' beginnen
# und den vorherigen Wert beibehält, falls sie nicht mit '-' beginnen, 
# mit Hilfe der forward fill Methode
daten['neue Spalte'] = daten['Werk'].apply(lambda x: x if x.startswith('-') else None)
daten['neue Spalte'].ffill(inplace=True)

# Entfernen des ersten Zeichens (vermutlich '-') und führender Leerzeichen in 'neue Spalte'
daten['neue Spalte'] = daten['neue Spalte'].str[1:].str.strip()

# Aufteilen der 'neue Spalte' in 'material' und 'kurztext' basierend auf dem ersten Leerzeichen
daten[['material', 'kurztext']] = daten['neue Spalte'].str.split(' ', n=1, expand=True)

# Verschieben der neuen Spalten an den Anfang des DataFrames
cols = daten.columns.tolist()
cols = cols[-3:] + cols[:-3]
daten = daten[cols]


# Umschmelzen des DataFrames, um ein langes Format zu erhalten
daten = daten[~daten['Werk'].str.startswith('-')]
daten = daten.melt(id_vars=daten.columns[:4], var_name='Month', value_name='Absatz')

# Extrahieren der letzten 6 Zeichen der 'Month'-Spalte für das Monatsformat
daten['Month'] = daten['Month'].str[-6:]
daten[['Werksnummer','Werksname']]=daten['Werk'].str.split(' ', n=1, expand=True)
daten.drop(['Werk'], axis=1, inplace=True)


# Zurücksetzen des Indexes, um nach dem Löschen von Zeilen einen sauberen Index zu haben
daten.reset_index(drop=True, inplace=True)
daten.drop(['neue Spalte'], axis=1,inplace=True)
cols = daten.columns.tolist()
cols = cols[-2:] + cols[:-2]
daten = daten[cols]

#Nan mit 0 auffüllen
daten['Absatz']=daten['Absatz'].fillna(0)
daten.head()


C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_5412\564491778.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daten['neue Spalte'].ffill(inplace=True)


,Werksnummer,Werksname,material,kurztext,Month,Absatz
0,0011,VILLEROY & BOCH AG,560050R1P,Subway 2.0 WWC for ViClean packed,Jan.22,0.0
1,0028,Villeroy&Boch Trading Shenzhen,560050R1P,Subway 2.0 WWC for ViClean packed,Jan.22,0.0
2,0042,Villeroy&Boch Trading Shanghai,560050R1P,Subway 2.0 WWC for ViClean packed,Jan.22,546.0
3,0011,VILLEROY & BOCH AG,61630001,Loop & Friends Undercounter Washbasin,Jan.22,345.0
4,0028,Villeroy&Boch Trading Shenzhen,61630001,Loop & Friends Undercounter Washbasin,Jan.22,0.0


In [7]:
daten = pd.read_excel(excel, 'IO', header=1)
io_df(daten).tail()

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_5412\7185798.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


,Material,Werk,Month,value
1843,UBQ170OBE2V-01,0042,Dec.23,18.0
1844,UBQ170OBE2V-01,0058,Dec.23,16.0
1845,UBQ170OBE2V-01,0078,Dec.23,316.0
1846,UBQ170OBE2V-01,0099,Dec.23,0.0
1847,FG0603809,0054,Dec.23,1190.0


## Gross Demand


In [13]:
def gross_demand_df(df:pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={'DC (Gross Demand after set & network drilldown) all DC': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 20', 'TOTAL'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    # Clear lines where 'Total' appears in the Werk column
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Gross Demand')
    df['Werk'] = df['Werk'].astype(int)
    df['Material'] = df['Material'].str[1:]
    return df

gross_demand_df(pd.read_excel(excel,'Gross Demand'))

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_5412\3594441.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


,Material,Werk,Month,Gross Demand
0,92219900,11,Feb.24,5950.370117
1,92219900,58,Feb.24,893.669983
2,92219900,99,Feb.24,764.919983
3,92219900,47,Feb.24,2.580000
4,92219900,42,Feb.24,4.000000
...,...,...,...,...
1021,92218068,47,Jul.25,0.000000
1022,92218068,42,Jul.25,0.000000
1023,92218068,99,Jul.25,0.000000
1024,92218068,58,Jul.25,0.000000


# Alte Bestellübersicht

In [379]:

excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung.xlsx')
bestell_df=pd.read_excel(excel,'Bestellübersicht')
bestell_df.head()

,Name des Lieferanten,Werk,Einkaufsbeleg,Position,Belegdatum,Material,Kurztext,BestellpreisME,Bestellmenge,noch zu liefern (Menge)
0,99011 Villeroy & Boch AG DC,47,49384564,1,2024-01-31,5614R001P,Subway 2.0 WWC DirectFlush verpackt,ST,24,0
1,99099 Villeroy & Boch KFT DC,57,49384528,3,2024-01-30,92219900,Fixation Set WWC Suprafix 3.0,ST,5000,0
2,90315 PRESSALIT A/S,11,57783718,1,2024-01-30,9M78S101,Subway 2.0 WC Sitz softcl/quick-rel.,ST,900,0
3,99011 Villeroy & Boch AG DC,42,49384482,18,2024-01-29,560050R1P,Subway 2.0 WWC for ViClean packed,ST,12,12
4,99078 UCOSAN B.V.,58,49384488,1,2024-01-29,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,ST,8,0


In [380]:
bestell_df.groupby('Material')['Kurztext'].unique().head()
material_dim = bestell_df.groupby('Material')['Kurztext'].unique()
material_dim = pd.DataFrame(material_dim)
material_dim = material_dim.reset_index()
material_dim.head(100)

,Material,Kurztext
0,560050R1P,"[Subway 2.0 WWC for ViClean packed, Subway 2.0..."
1,5614R001P,"[Subway 2.0 WWC DirectFlush verpackt, Subway 2..."
2,61630001,[Loop & Friends Unterbau-WB 540x345mm]
3,92218068,"[ViConnect 300S Betätigungs Pl. dual fl, ViCon..."
4,92219900,"[Fixation Set WWC Suprafix 3.0, Befestigung WW..."
5,92249068,[ViConnect 200S Betätigungs Pl. dual fl]
6,9M78S101,"[Subway 2.0 WC Sitz softcl/quick-rel., Subway ..."
7,TVS00035100000,[Univ. Vi-Box]
8,TVS10535311061,"[O.novo Start conc. mixer, O.novo Start Unterp..."
9,UBQ170OBE2V-01,[Oberon bathtub 170x75 rect]


In [381]:
#Dimensionstabelle Material fertig
material_me = bestell_df.groupby('Material')['BestellpreisME'].unique().apply(lambda arr: arr[0] if arr.size > 0 else "").reset_index(name='BestellpreisME')

material_dim['BestellpreisME']=material_me['BestellpreisME']


material_dim.head(100)

,Material,Kurztext,BestellpreisME
0,560050R1P,"[Subway 2.0 WWC for ViClean packed, Subway 2.0...",ST
1,5614R001P,"[Subway 2.0 WWC DirectFlush verpackt, Subway 2...",ST
2,61630001,[Loop & Friends Unterbau-WB 540x345mm],ST
3,92218068,"[ViConnect 300S Betätigungs Pl. dual fl, ViCon...",ST
4,92219900,"[Fixation Set WWC Suprafix 3.0, Befestigung WW...",ST
5,92249068,[ViConnect 200S Betätigungs Pl. dual fl],ST
6,9M78S101,"[Subway 2.0 WC Sitz softcl/quick-rel., Subway ...",ST
7,TVS00035100000,[Univ. Vi-Box],ST
8,TVS10535311061,"[O.novo Start conc. mixer, O.novo Start Unterp...",ST
9,UBQ170OBE2V-01,[Oberon bathtub 170x75 rect],ST


In [382]:
#Spalten droppen die nicht gebraucht werden
bestell_df.drop(['Kurztext','BestellpreisME'],axis=1,inplace=True)

bestell_df

,Name des Lieferanten,Werk,Einkaufsbeleg,Position,Belegdatum,Material,Bestellmenge,noch zu liefern (Menge)
0,99011 Villeroy & Boch AG DC,47,49384564,1,2024-01-31,5614R001P,24,0
1,99099 Villeroy & Boch KFT DC,57,49384528,3,2024-01-30,92219900,5000,0
2,90315 PRESSALIT A/S,11,57783718,1,2024-01-30,9M78S101,900,0
3,99011 Villeroy & Boch AG DC,42,49384482,18,2024-01-29,560050R1P,12,12
4,99078 UCOSAN B.V.,58,49384488,1,2024-01-29,UBQ170OBE2V-01,8,0
...,...,...,...,...,...,...,...,...
873,99011 Villeroy & Boch AG DC,47,49345796,2,2022-07-05,92219900,20,0
874,99011 Villeroy & Boch AG DC,47,49345796,3,2022-07-05,92249068,28,0
875,0005 Villeroy & Boch Magyarország K,99,48175634,4,2022-07-04,92219900,5000,0
876,99011 Villeroy & Boch AG DC,47,49345653,1,2022-07-04,5614R001P,36,0


In [383]:
bestell_df[['Lieferantennummer','Lieferantenname']]=bestell_df['Name des Lieferanten'].str.split(' ',n=1,expand=True)

bestell_df.drop(['Name des Lieferanten'],axis=1,inplace=True)
first_column = bestell_df.pop('Lieferantennummer')
second_column = bestell_df.pop('Lieferantenname')  

#Reihenfolge angepasst
bestell_df.insert(0, 'Lieferantennummer', first_column) 
bestell_df.insert(1, 'Lieferantenname', second_column) 
# Ergebnis anzeigen
print(bestell_df.head(100))

   Lieferantennummer                 Lieferantenname  Werk  Einkaufsbeleg  \
0              99011           Villeroy & Boch AG DC    47       49384564   
1              99099          Villeroy & Boch KFT DC    57       49384528   
2              90315                   PRESSALIT A/S    11       57783718   
3              99011           Villeroy & Boch AG DC    42       49384482   
4              99078                     UCOSAN B.V.    58       49384488   
..               ...                             ...   ...            ...   
95             99011           Villeroy & Boch AG DC    47       49382138   
96             99028         Villeroy & Boch Trading    11       49382160   
97            158995        Guangdong Huayi Plumbing    28       57772454   
98            158995        Guangdong Huayi Plumbing    28       57772454   
99              0005  Villeroy & Boch Magyarország K    99       48185757   

    Position Belegdatum        Material  Bestellmenge  noch zu liefern (Men

# Ab hier die IO Tabelle

In [400]:
from datetime import datetime
import re
#Daten einlesen und NaN Wert mit 0 Werten auffüllen
io_df=pd.read_excel(excel,'IO')
io_df.fillna(0,inplace=True)
io_df.columns=io_df.iloc[0]
io_df.drop(0,axis=0,inplace=True)

def string_to_timestamp(input_string_array):
    all_times = []
    for item in input_string_array:
        # Check if the item is a string and has a sufficient length to contain month and year
        if isinstance(item, str) and len(item) > 6:
            # The last 6 characters are expected to be in the format "MMM.YY"
            clean_string = item[-6:]
            month = clean_string[:3]
            year = int('20' + clean_string[-2:])
            
            # Create a Period object for the month and year
            time_index = pd.Period(f'{year}-{month}', freq='M')
            all_times.append(time_index)
    return all_times
# print(io_df.columns[1:][0][-6:])

# print(string_to_timestamp(io_df.columns[1:]))

#letzten 2 Zeilen löschen
io_df=io_df.iloc[0:-2]

#alle vorangestellten Leerzeichen löschen
io_df['Year']=io_df['Year'].str.lstrip()


# Überprüfen Sie, ob die 'Year'-Spalte mit einem Bindestrich beginnt
filtered_df = list(io_df[io_df['Year'].astype(str).str.startswith('-')]['Year'])
total_df = list(io_df[io_df['Year'].astype(str).str.contains('Total')]['Year'])
# print(io_df.index[io_df['Year'] =='Total 92219900 Fixation Set WWC Suprafix 3.0'])

def io_preprocessed(df,artikel, total,times):
    fertig_df=pd.DataFrame()
    time_list=string_to_timestamp(df.columns[1:0])
    for i in range(artikel):











    pass







Index([69], dtype='int64')


In [ ]:
print(total_df)

['Total 560050R1P Subway 2.0 WWC for ViClean packed', 'Total 61630001 Loop & Friends Undercounter Washbasin', 'Total 5614R001P Subway 2.0 WWC DirectFlush packed', 'Total 9M78S101 Subway 2.0 WC Seat softcl/quick-rel.', 'Total 92249068 ViConnect 200S flush plate dual fl', 'Total 92218068 ViConnect 300S flush plate dual fl', 'Total 92219900 Fixation Set WWC Suprafix 3.0', 'Total TVS00035100000 Univ. Vi-Box', 'Total TVS10535311061 O.novo Start conc. mixer', 'Total UBQ170OBE2V-01 Oberon bathtub 170x75 rect', 'Total FG0603809 Vatette 1219 Blanking-Pc 10']


In [418]:
aaa=pd.read_excel(excel,'Absatz')

def convert_to_datetime(date_str):
    de_to_en_month_abbr = {
    'JAN': 'Jan',  # Januar
    'FEB': 'Feb',  # Februar
    'MAR': 'Mar',  # März
    'APR': 'Apr',  # April
    'MAI': 'May',  # Mai
    'JUN': 'Jun',  # Juni
    'JUL': 'Jul',  # Juli
    'AUG': 'Aug',  # August
    'SEP': 'Sep',  # September
    'OKT': 'Oct',  # Oktober
    'NOV': 'Nov',  # November
    'DEZ': 'Dec'   # Dezember
    
    }
    
    month = str.upper(date_str[:3])
    year = int(date_str[-4:])
            
    return pd.Period(f'{year}-{de_to_en_month_abbr[month]}', freq='M')


def absatz_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.iloc[1:]
    df.drop(['Unnamed: 1', 'Geschäftsj./Periode', 'Gesamtergebnis'], axis=1, inplace=True)
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Artikel', 'Werk'], var_name='Month', value_name='Absatz') 
    df['Werk']=df['Werk'].apply(int) 
    df['Month'] = df['Month'].apply(convert_to_datetime)
    return df

print(absatz_df(aaa))
print(convert_to_datetime('Jan 2022'))

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_12280\24987988.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 1', 'Geschäftsj./Periode', 'Gesamtergebnis'], axis=1, inplace=True)
C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_12280\24987988.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


             Artikel  Werk    Month  Absatz
0          560050R1P    11  2022-01     0.0
1          560050R1P    28  2022-01     0.0
2          560050R1P    42  2022-01   490.0
3           61630001    11  2022-01   579.0
4           61630001    28  2022-01     0.0
...              ...   ...      ...     ...
1920  UBQ170OBE2V-01    42  2024-01     4.0
1921  UBQ170OBE2V-01    58  2024-01     9.0
1922  UBQ170OBE2V-01    78  2024-01   424.0
1923  UBQ170OBE2V-01    99  2024-01     0.0
1924       FG0603809    54  2024-01  2180.0

[1925 rows x 4 columns]
2022-01


In [ ]:
def convert_to_datetime(date_str):
    de_to_en_month_abbr = {
    'JAN': 'Jan',  # Januar
    'FEB': 'Feb',  # Februar
    'MAR': 'Mar',  # März
    'APR': 'Apr',  # April
    'MAI': 'May',  # Mai
    'JUN': 'Jun',  # Juni
    'JUL': 'Jul',  # Juli
    'AUG': 'Aug',  # August
    'SEP': 'Sep',  # September
    'OKT': 'Oct',  # Oktober
    'NOV': 'Nov',  # November
    'DEZ': 'Dec'   # Dezember
    
    }
    
    month = str.upper(date_str[:3])
    year = int(date_str[-4:])
    if month not in de_to_en_month_abbr.values():        
        return pd.Period(f'{year}-{de_to_en_month_abbr[month]}', freq='M')
    else:
        return pd.Period(f'{year}-{month}', freq='M')


def io_df(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={'Year': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 25'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='value')
    df['Material'] = df['Material'].str.split().str[1]
    df['Werk'] = df['Werk'].str.split().str[0]
    df['Month'] = df['Month'].str[-6:]
    df['Month'].apply(lambda s: s[0:4]+'20'+s[-2:])
    
    return df



aaa=pd.read_excel(excel,'IO',header=1)

print(io_df(aaa))

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_12280\940514211.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


: 

In [435]:
s='Jan.22'
s=s[0:4]+'20'+s[-2:]
print(s[-4: ])




2022


In [ ]:
def convert_to_datetime(date_str):
    de_to_en_month_abbr = {
    'JAN': 'Jan',  # Januar
    'FEB': 'Feb',  # Februar
    'MAR': 'Mar',  # März
    'APR': 'Apr',  # April
    'MAI': 'May',  # Mai
    'JUN': 'Jun',  # Juni
    'JUL': 'Jul',  # Juli
    'AUG': 'Aug',  # August
    'SEP': 'Sep',  # September
    'OKT': 'Oct',  # Oktober
    'NOV': 'Nov',  # November
    'DEZ': 'Dec'   # Dezember
    
    }
    
    month = str.upper(date_str[:3])
    year = int(date_str[-4:])
    if month not in de_to_en_month_abbr.values():        
        return pd.Period(f'{year}-{de_to_en_month_abbr[month]}', freq='M')
    else:
        return pd.Period(f'{year}-{month}', freq='M')


def io_df(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={'Year': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 25'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='value')
    df['Material'] = df['Material'].str.split().str[1]
    df['Werk'] = df['Werk'].str.split().str[0]
    df['Month'] = df['Month'].str[-6:]

    
    return df



aaa=pd.read_excel(excel,'IO',header=1)

print(io_df(aaa))

## Bestandsdatei

In [125]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Bestand 012019 - 022024.xlsx')
bestand_df=pd.read_excel(excel,'YLOB0003T - 2024-03-13T162035.7',header=14)
bedingung_st=bestand_df.iloc[0]=='ST'
bedingung_eur=bestand_df.iloc[0]=='EUR'
# Maske für nur die Mengen und Bruttowert
mengen_ST=bestand_df.loc[:,bedingung_st]
mengen_ST=mengen_ST.iloc[2:]

bruttowert_eur=bestand_df.loc[:,bedingung_eur]
bruttowert_eur=bruttowert_eur.iloc[2:]
cols=bestand_df.columns.to_list()
cols[0]='Artikel'
cols[1]='Kurztext'
cols[2]='Werk'
cols[3]='Name des Werks'
bestand_df.columns=cols
bestand_df=bestand_df.iloc[2:,:4]
bestand_menge=pd.concat([bestand_df,mengen_ST],axis= 1)
#Dataframes in richtige Form bringen
bestand_menge=bestand_menge.melt(id_vars=bestand_menge.columns[:4],var_name='Month', value_name='Menge in ST')
bruttowert_eur=bruttowert_eur.melt(var_name='Variable', value_name='Bruttowert in EUR').drop(['Variable'],axis=1)

final_df= pd.concat([bestand_menge,bruttowert_eur],axis=1)
final_df=final_df.fillna({"Menge in ST": 0, "Bruttowert in EUR": 0})

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_9084\842395024.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df=final_df.fillna({"Menge in ST": 0, "Bruttowert in EUR": 0})


In [126]:
final_df.head()

,Artikel,Kurztext,Werk,Name des Werks,Month,Menge in ST,Bruttowert in EUR
0,560050R1P,Subway 2.0 WWC for V,0011,VILLEROY & BOCH AG,01.2019,0,0.00
1,560050R1P,Subway 2.0 WWC for V,0028,Villeroy&Boch Trading Shenzhen,01.2019,0,0.00
2,560050R1P,Subway 2.0 WWC for V,0041,Villeroy & Boch OOO,01.2019,0,0.00
3,560050R1P,Subway 2.0 WWC for V,0042,Villeroy&Boch Trading Shanghai,01.2019,1774,178802.06
4,61630001,Loop & Friends Unter,0011,VILLEROY & BOCH AG,01.2019,2733,140257.56


## Lagerkosten je DC

In [150]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lagerkosten je DC.xlsx')
lagerkosten_df=pd.read_excel(excel,'Lagerkosten je Standort', header=1)
lagerkosten_df=lagerkosten_df.iloc[0:8,1:]
lagerkosten_df=lagerkosten_df.rename(columns={"Unnamed: 1": "Standort"})
lagerkosten_df['DC']=lagerkosten_df['DC'].apply(int)
lagerkosten_df['DC']=lagerkosten_df['DC'].apply(lambda x: "00"+str(x))

In [151]:
lagerkosten_df.head(100)

,Standort,DC,Storage cost / pallet or m² / month
0,Losheim,0011,4.100000
1,Treuchtlingen,0067,6.700000
2,HMV,0099,3.500000
3,Lugoj,0058,3.680417
4,Valence d'Agen,0044,9.125000
5,Saraburi,0049,6.326667
6,Shanghai,0042,5.733542
7,Shenzhen,0028,5.535833


## Menge Pro Palette

In [152]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lagerkosten je DC.xlsx')
pallete_df=pd.read_excel(excel,'Menge pro Palette')

In [153]:
pallete_df.head()

,Article,Quantity per Pallet (DIN-Format)
0,GB1921900201,99999
1,GB1921100240,90000
2,GB1921100005,80000
3,TVP00000100400,64000
4,TVP00000521000,64000


## Lieferzeiten und Lieferwege DC-Netzwerk

In [6]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lieferzeiten und Lieferwege DC-Netzwerk.xlsx')
lieferwege_df=pd.read_excel(excel,'interne Lieferwege')

In [7]:
lieferwege_df.head()

,Source DC,DC,Transshipment DC 1,Transshipment DC 2,Transshipment DC 3,Unnamed: 5
0,5,0001,NaN,NaN,NaN,
1,5,0005,NaN,NaN,NaN,
2,5,0007,NaN,NaN,NaN,
3,5,0008,NaN,NaN,NaN,
4,5,0009,NaN,NaN,NaN,


In [22]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lieferzeiten und Lieferwege DC-Netzwerk.xlsx')
lieferzeiten_df=pd.read_excel(excel,'Lieferzeiten_gefiltert')
lieferzeiten_df=lieferzeiten_df.iloc[:,:3]
cols=lieferzeiten_df.columns.to_list()
cols[2]="Lieferzeiten in d"
lieferzeiten_df.columns=cols


In [24]:
lieferzeiten_df.tail()

,Source DC,DC,Lieferzeiten in d
51,78,99,15
52,99,11,9
53,99,46,15
54,99,47,15
55,99,58,8


## Standardkosten & Lieferzeit

In [25]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Standardkosten &-lieferzeit.xlsx')
standardkosten_df=pd.read_excel(excel,0)

In [27]:
standardkosten_df.tail()

,Artikel,Standardkosten (€),Standardbereitstellungszeit (Arbeitstage)
7,FG0603809,0.16,1
8,FP603476,13.41,10
9,TVS00035100000,14.85,1
10,TVS10535311061,24.89,1
11,UBQ170OBE2V-01,173.92,1


## Verfügbare Mengen pro DC

In [55]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')
file=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')


def data_man(werk:str,number_of_sheet:int):
    menge_dc_df=pd.read_excel(excel,number_of_sheet)
    #alle Spalten führende Leerzeichen wegmachen
    cols=menge_dc_df.columns.to_list()
    cols=[str.strip(x) for x in cols]
    menge_dc_df.columns=cols
    #alle unnamed Spalten raus
    menge_dc_df = menge_dc_df.loc[:, ~menge_dc_df.columns.str.contains('^Unnamed')]
    menge_dc_df.drop(['Summe:'], axis=1,inplace= True)
    menge_dc_df=menge_dc_df.iloc[:-2]
    menge_dc_df=menge_dc_df.melt(id_vars=menge_dc_df.columns[0],var_name='Material',value_name='Menge')
    menge_dc_df[['Material','Kurztext']]=menge_dc_df['Material'].str.split(" ",n=1,expand=True)
    cols=menge_dc_df.columns.to_list()
    #Reihenfolge der Spalten angepasst
    menge_dc_df=menge_dc_df[cols[0:2]+cols[-1:]+cols[2:3]]
    menge_dc_df['Werk']=werk
    
    return menge_dc_df


final_df=pd.DataFrame()
for i in range(0,len(file.sheet_names)):
    data=data_man(file.sheet_names[i],i)
    final_df=pd.concat([final_df,data],axis=0)
final_df.reset_index(drop=True,inplace=True)



,Monat,Material,Kurztext,Menge,Werk
5339,2023-10-01,TVS10535311061,O.novo Start Unterputz Armatur,0.0,DC 0501 (n.r.)
5340,2023-11-01,TVS10535311061,O.novo Start Unterputz Armatur,0.0,DC 0501 (n.r.)
5341,2023-12-01,TVS10535311061,O.novo Start Unterputz Armatur,0.0,DC 0501 (n.r.)
5342,2024-01-01,TVS10535311061,O.novo Start Unterputz Armatur,0.0,DC 0501 (n.r.)
5343,2024-02-01,TVS10535311061,O.novo Start Unterputz Armatur,0.0,DC 0501 (n.r.)


## Backlog ein lesen

In [3]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')
file=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')



['Prod 0005', 'Prod 0072', '0054', '0071', 'DC 0011', 'DC 0028', 'DC 0042', 'DC 0047', 'DC 0049', 'DC 0058', 'DC 0078', 'DC 0099', 'DC 0025 (n.r.)', 'DC 0041 (n.r.)', 'DC 0063 (n.r.)', 'DC 0064 (n.r.)', 'DC 0065 (n.r.)', 'DC 0501 (n.r.)']


In [70]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Backlog 012019 - 022024.xlsx')
backlog_blocked_df=pd.read_excel(excel,'wo blocked',header=1)
backlog_blocked_df=backlog_blocked_df.iloc[1:]
backlog_blocked_df.reset_index(drop=True, inplace=True)
cols=backlog_blocked_df.columns.to_list()
cols[0],cols[1],cols[2],cols[3]='Material und Werk','Material','Kurztext','Werk'
backlog_blocked_df.columns=cols
backlog_blocked_df['Werk']=backlog_blocked_df['Werk'].apply(lambda x: "00" +str(int(x)))
backlog_blocked_df=backlog_blocked_df.melt(id_vars=backlog_blocked_df.columns[:4],var_name='Month', value_name='Menge')

In [73]:
backlog_blocked_df.head()

,Material und Werk,Material,Kurztext,Werk,Month,Menge
0,560050R1P0028,560050R1P,Subway 2.0 WWC for V,0028,2024-03-01 00:00:00,0.0
1,560050R1P0042,560050R1P,Subway 2.0 WWC for V,0042,2024-03-01 00:00:00,0.0
2,616300010011,61630001,Loop & Friends Unter,0011,2024-03-01 00:00:00,0.0
3,616300010028,61630001,Loop & Friends Unter,0028,2024-03-01 00:00:00,0.0
4,616300010042,61630001,Loop & Friends Unter,0042,2024-03-01 00:00:00,-47.0


In [107]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Backlog 012019 - 022024.xlsx')
backlog_blocked_df=pd.read_excel(excel,'incl. blocked',header=1)
backlog_blocked_df=backlog_blocked_df.iloc[1:]
backlog_blocked_df.dropna(inplace=True)
backlog_blocked_df.reset_index(drop=True, inplace=True)
cols=backlog_blocked_df.columns.to_list()
cols[0],cols[1],cols[2],cols[3]='Material und Werk','Material','Kurztext','Werk'
backlog_blocked_df.columns=cols
backlog_blocked_df['Werk']=backlog_blocked_df['Werk'].apply(int)
backlog_blocked_df=backlog_blocked_df.melt(id_vars=backlog_blocked_df.columns[:4],var_name='Month', value_name='Menge')



In [108]:
backlog_blocked_df.tail()

,Material und Werk,Material,Kurztext,Werk,Month,Menge
3712,922180680007,92218068,ViConnect 300S Betät,7,2019-01-01 00:00:00,0.0
3713,922180680011,92218068,ViConnect 300S Betät,11,2019-01-01 00:00:00,0.0
3714,922180680063,92218068,ViConnect 300S Betät,63,2019-01-01 00:00:00,0.0
3715,TVS000351000000501,TVS00035100000,Univ. Vi-Box,501,2019-01-01 00:00:00,0.0
3716,FG06038090054,FG0603809,Vatette 1219 Blanking-Pc 10,54,2019-01-01 00:00:00,0.0


In [109]:
excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Backlog 012019 - 022024.xlsx')
backlog_blocked_df=pd.read_excel(excel,'wo blocked',header=1)
backlog_blocked_df=backlog_blocked_df.iloc[1:]
backlog_blocked_df.dropna(inplace=True)
backlog_blocked_df.reset_index(drop=True, inplace=True)
cols=backlog_blocked_df.columns.to_list()
cols[0],cols[1],cols[2],cols[3]='Material und Werk','Material','Kurztext','Werk'
backlog_blocked_df.columns=cols
backlog_blocked_df['Werk']=backlog_blocked_df['Werk'].apply(int)
backlog_blocked_df=backlog_blocked_df.melt(id_vars=backlog_blocked_df.columns[:4],var_name='Month', value_name='Menge')

In [111]:
backlog_blocked_df.tail()

,Material und Werk,Material,Kurztext,Werk,Month,Menge
3712,922180680007,92218068,ViConnect 300S Betät,7,2019-01-01 00:00:00,0.0
3713,922180680011,92218068,ViConnect 300S Betät,11,2019-01-01 00:00:00,0.0
3714,922180680063,92218068,ViConnect 300S Betät,63,2019-01-01 00:00:00,0.0
3715,TVS000351000000501,TVS00035100000,Univ. Vi-Box,501,2019-01-01 00:00:00,0.0
3716,FG06038090054,FG0603809,Vatette 1219 Blanking-Pc 10,54,2019-01-01 00:00:00,0.0


# Alle Funktionen testen und Kleinigkeiten optimieren

## Absatz Tabelle

In [134]:
from datetime import datetime
def convert_to_datetime(date_str):
    de_to_en_month_abbr = {
    'JAN': 'Jan',  # Januar
    'FEB': 'Feb',  # Februar
    'MAR': 'Mar',  # März
    'APR': 'Apr',  # April
    'MAI': 'May',  # Mai
    'JUN': 'Jun',  # Juni
    'JUL': 'Jul',  # Juli
    'AUG': 'Aug',  # August
    'SEP': 'Sep',  # September
    'OKT': 'Oct',  # Oktober
    'NOV': 'Nov',  # November
    'DEZ': 'Dec'   # Dezember
    
    }
    
    month = str.upper(date_str[:3])
    year = int(date_str[-4:])
    if month in de_to_en_month_abbr:
        month=de_to_en_month_abbr[month]
    date_str=f'{month} {year}'
    return pd.to_datetime(datetime.strptime(date_str, '%b %Y'))

def absatz_neu_df(path:str) -> pd.DataFrame: 
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Absatz 012019 - 022024.xlsx')
    df=pd.read_excel(excel,0)
    df=df.iloc[12:]
    df.reset_index(drop=True, inplace=True)
    df.columns=df.iloc[0]
    df.drop(0,inplace=True)
    cols=df.columns.tolist()
    cols[0]=df.iloc[0,0]
    cols[1]='Kurztext'
    cols[2]=df.iloc[0,2]
    cols[3]='Gesellschaft'
    df.columns=cols
    df=df.iloc[2:]
    menge_df=df.iloc[:,:66]
    erlöse_df=df.iloc[:,66:128]
    nu_df=df.iloc[:,128:]

    menge_df=menge_df.melt(id_vars=menge_df.columns[0:4], var_name='Month', value_name='Menge')
    erlöse_df=erlöse_df.melt(var_name='Month', value_name='Erlös')
    erlöse_df.drop(['Month'],axis=1, inplace= True)
    nu_df=nu_df.melt(var_name='Month', value_name='NU')
    nu_df.drop(['Month'],axis=1, inplace= True)

    final_df=pd.concat([menge_df,erlöse_df,nu_df],axis= 1)
    final_df[['Menge','Erlös','NU']]=final_df[['Menge','Erlös','NU']].fillna(0)
    final_df['Month']=final_df['Month'].apply(convert_to_datetime)
    return final_df

new_df=absatz_neu_df('sds')
new_df.head()


C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_3088\1876141290.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df[['Menge','Erlös','NU']]=final_df[['Menge','Erlös','NU']].fillna(0)


,Artikel,Kurztext,Werk,Gesellschaft,Month,Menge,Erlös,NU
0,560050R1P,Subway 2.0 WWC for V,0011,VILLEROY & BOCH AG,2019-01-01,0,0.000000,0.00
1,560050R1P,Subway 2.0 WWC for V,0028,Villeroy&Boch Tradin,2019-01-01,0,0.000000,0.00
2,560050R1P,Subway 2.0 WWC for V,0041,Villeroy & Boch OOO,2019-01-01,0,0.000000,0.00
3,560050R1P,Subway 2.0 WWC for V,0042,Villeroy&Boch Tradin,2019-01-01,101,198.828972,20081.73
4,61630001,Loop & Friends Unter,0007,VILLEROY & BOCH AG,2019-01-01,0,0.000000,0.00


## Backlog Tabellen

In [49]:
from datetime import datetime
def werk_str(x):
    x=str(x)
    length_werk=len(x)
    diff=4-length_werk
    final_werk= diff*'0'+x
    return final_werk

def backlog_blocked_df(path:str,sheetname:str):
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Backlog 012019 - 022024.xlsx')
    backlog_blocked_df=pd.read_excel(excel,sheetname,header=1)
    backlog_blocked_df=backlog_blocked_df.iloc[1:]
    backlog_blocked_df.dropna(inplace=True)
    backlog_blocked_df.reset_index(drop=True, inplace=True)
    cols=backlog_blocked_df.columns.to_list()
    cols[0],cols[1],cols[2],cols[3]='Material und Werk','Material','Kurztext','Werk'
    backlog_blocked_df.columns=cols
    backlog_blocked_df['Werk']=backlog_blocked_df['Werk'].apply(int)
    backlog_blocked_df=backlog_blocked_df.melt(id_vars=backlog_blocked_df.columns[:4],var_name='Month', value_name='Menge')
    #Datum im richtigen Format
    backlog_blocked_df['Month']=backlog_blocked_df['Month'].apply(pd.to_datetime)
    #Ursprünglichen String wieder generiert
    backlog_blocked_df['Werk']=backlog_blocked_df['Werk'].apply(werk_str)
    return backlog_blocked_df

df=backlog_blocked_df('ss','wo blocked')
df.tail()

,Material und Werk,Material,Kurztext,Werk,Month,Menge
3712,922180680007,92218068,ViConnect 300S Betät,0007,2019-01-01,0.0
3713,922180680011,92218068,ViConnect 300S Betät,0011,2019-01-01,0.0
3714,922180680063,92218068,ViConnect 300S Betät,0063,2019-01-01,0.0
3715,TVS000351000000501,TVS00035100000,Univ. Vi-Box,0501,2019-01-01,0.0
3716,FG06038090054,FG0603809,Vatette 1219 Blanking-Pc 10,0054,2019-01-01,0.0


## Beispielartikel_Bestandoptimierung Tabelle

In [2]:
def bestelluebersicht_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    bestell_df=pd.read_excel(excel,sheetname)
    #bestell_df.drop(['Kurztext','BestellpreisME'],axis=1,inplace=True) -> muss erstmal nicht sein
    bestell_df[['Lieferantennummer','Lieferantenname']]=bestell_df['Name des Lieferanten'].str.split(' ',n=1,expand=True)

    bestell_df.drop(['Name des Lieferanten'],axis=1,inplace=True)
    #Reihenfolge der Columns angepasst
    first_column = bestell_df.pop('Lieferantennummer')
    second_column = bestell_df.pop('Lieferantenname')  

    bestell_df.insert(0, 'Lieferantennummer', first_column) 
    bestell_df.insert(1, 'Lieferantenname', second_column) 

    #alle Leerzeichen am Anfang werden gelöscht
    bestell_df['Lieferantenname']=bestell_df['Lieferantenname'].apply(str.lstrip)
    bestell_df['Belegdatum']=bestell_df['Belegdatum'].apply(pd.to_datetime)
    bestell_df['Werk']=bestell_df['Werk'].apply(werk_str)
    return bestell_df

df=bestelluebersicht_df('sda','BestÜ neu')
df.head()

NameError: name 'werk_str' is not defined

## EKKO Tabelle

In [53]:
def ekko_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)
    df['Beleg_Anlege_Dat'] = df['Angel.am']
    df.drop(['Angel.am', 'BelegDat'], axis=1, inplace=True)
    return df

df=ekko_df('sd','EKKO neu')
df.head()


,EinkBeleg,T,Art,L,Angel.von,Lieferant,LWk,IncTm,Beleg_Anlege_Dat
0,48159323,F,MPO,NaN,JUNGGY,99005.0,5.0,EXW,2020-10-14
1,48160760,F,UB,NaN,BURGN,99072.0,72.0,EXW,2020-11-30
2,48161400,F,MPO,NaN,MARKIEWICZJ,99010.0,10.0,EXW,2021-01-04
3,48161554,F,UB,NaN,BURGN,99072.0,72.0,EXW,2021-01-07
4,48161555,F,UB,NaN,BURGN,99072.0,72.0,EXW,2021-01-07


## EKPO Tabelle

In [61]:
def ekpo_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)
    df['Anlegedatum']=df['Anlegedatum'].apply(pd.to_datetime)
    df['Werk']=df['Werk'].apply(werk_str)
    return df

df=ekpo_df('sds','EKPO neu')
df.head()

,EinkBeleg,Pos,Material,Werk,LOrt,Bestellmenge,BME,WEZ,ELi,WE,Anlegedatum
0,48159323,1,7113F001,0099,NaN,60.0,ST,0,X,X,2020-10-14
1,48159323,2,7113F001,0099,NaN,60.0,ST,0,X,X,2020-10-14
2,48159323,3,7113F001,0099,NaN,60.0,ST,0,X,X,2020-10-14
3,48159323,4,7113F001,0099,NaN,60.0,ST,0,X,X,2020-10-14
4,48159323,5,7113F001,0099,NaN,60.0,ST,0,X,X,2020-10-14


## EKBE Tabelle

In [66]:
def ekbe_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)    
    # df = df[df['BTy'] == 'E']
    # df = df[(df['BwA'] == 101.) | (df['BwA'] == 109.)]  
    df['Buch.dat.']=df['Buch.dat.'].apply(pd.to_datetime)
    return df

df= ekbe_df('sds','EKBE neu')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85914 entries, 0 to 85913
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   EinkBeleg  85914 non-null  int64         
 1   Pos        85914 non-null  int64         
 2   BwA        72303 non-null  float64       
 3   Buch.dat.  85914 non-null  datetime64[ns]
 4   Menge      85914 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 3.3 MB


## AFKO Tabelle

In [72]:
def afko_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)   
    df[['Eckendtermin','Eckstarttermin','Iststarttermin','Istendtermin']]=df[['Eckendtermin','Eckstarttermin','Iststarttermin','Istendtermin']].apply(pd.to_datetime)
    return df

df= afko_df('sds','AFKO neu')
df.head()


,Auftrag,Eckendtermin,Eckstarttermin,Material,Iststarttermin,Istendtermin,Gesamte Auftragsmenge
0,1500909609,2019-06-24,2019-06-20,FP603476,NaT,NaT,18340
1,1501124011,2021-02-01,2021-01-21,FP603476,2021-01-21,2021-01-22,124250
2,1501124012,2021-02-01,2021-01-22,FP603476,2021-01-27,2021-02-03,120000
3,1501124522,2021-03-25,2021-03-02,FP603476,NaT,NaT,303086
4,1501124563,2021-03-01,2021-02-12,FP603476,2021-02-12,2021-02-15,175100


## AUFK Tabelle

In [77]:
def aufk_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)   
    df['Werk']=df['Werk'].apply(werk_str)
    df['Erf.datum']=df['Erf.datum'].apply(pd.to_datetime)
    return df

df= aufk_df('sds','AUFK neu')
df.tail()    

,Auftrag,Erf.datum,Werk
76,1501423404,2023-09-13,0054
77,1501432500,2023-10-24,0054
78,1501435219,2023-11-07,0054
79,1501437950,2023-12-01,0054
80,1501452635,2024-01-09,0054


## MVER Tabelle

In [91]:


def mver_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df_1 = pd.read_excel(excel, 'MVER', nrows=74)
    df_2 = pd.read_excel(excel, 'MVER', skiprows=76, nrows=160-76)
    
    def change_month_to_monthYear(df: pd.DataFrame) -> pd.DataFrame:
        """
        Change the name of the columns to the format MonthYear
        """
        val = str(df.columns[0])[2:]
        for title in df.columns[1:]:
            df.rename(columns={title: title + '.' + val}, inplace=True)
        return df

    def change_first_three_columns(df: pd.DataFrame) -> pd.DataFrame:
        """
        Change the name of the first three columns to the values of the first row
        """
        for i in range(3):
            df.rename(columns={df.columns[i]: df.iloc[0, i]}, inplace=True)
        return df
    
    def convert_to_datetime_MVER(date_str):
        de_to_en_month_abbr = {
        'JAN': 'Jan',  # Januar
        'FEB': 'Feb',  # Februar
        'MAR': 'Mar',  # März
        'APR': 'Apr',  # April
        'MAI': 'May',  # Mai
        'JUN': 'Jun',  # Juni
        'JUL': 'Jul',  # Juli
        'AUG': 'Aug',  # August
        'SEP': 'Sep',  # September
        'OKT': 'Oct',  # Oktober
        'NOV': 'Nov',  # November
        'DEZ': 'Dec',   # Dezember
        'MRZ': 'Mar'
        }
        
        month = str.upper(date_str[:3])
        year = 2000+int(date_str[-2:])
        if month in de_to_en_month_abbr:
            month=de_to_en_month_abbr[month]
        date_str=f'{month} {year}'
        return pd.to_datetime(datetime.strptime(date_str, '%b %Y'))

    df_2 = change_month_to_monthYear(df_2)
    df_1 = change_month_to_monthYear(df_1)

    df_2 = change_first_three_columns(df_2)
    df_1 = change_first_three_columns(df_1)
    df_2 = df_2.iloc[1:]
    df_1 = df_1.iloc[1:]

    df_1.drop(['Jahr'], axis=1, inplace=True)
    df_2.drop(['Jahr'], axis=1, inplace=True)
    df_1 = pd.melt(df_1, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')
    df_2 = pd.melt(df_2, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')

    df_final= pd.concat([df_1,df_2], axis=0, ignore_index=True)

    # df = df_1.merge(df_2, on=['Material', 'Werk'], how='outer')
    # df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Absatz')
    df_final['Month']=df_final['Month'].apply(lambda x: x.replace('.',' '))
    df_final['Month']=df_final['Month'].apply(convert_to_datetime_MVER)
    return df_final

df=mver_df('sds','sd')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Material  1848 non-null   object        
 1   Werk      1848 non-null   object        
 2   Month     1848 non-null   datetime64[ns]
 3   Absatz    1848 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 57.9+ KB


## IO Tabelle

In [95]:
def io_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    daten=pd.read_excel(excel,sheetname,header=1)     
    daten.rename(columns={'Year': 'Werk'}, inplace=True)
    daten.drop(columns=['Unnamed: 25'], inplace=True)

    def convert_to_datetime_MVER(date_str):
        de_to_en_month_abbr = {
        'JAN': 'Jan',  # Januar
        'FEB': 'Feb',  # Februar
        'MAR': 'Mar',  # März
        'APR': 'Apr',  # April
        'MAI': 'May',  # Mai
        'JUN': 'Jun',  # Juni
        'JUL': 'Jul',  # Juli
        'AUG': 'Aug',  # August
        'SEP': 'Sep',  # September
        'OKT': 'Oct',  # Oktober
        'NOV': 'Nov',  # November
        'DEZ': 'Dec',   # Dezember
        'MRZ': 'Mar'
        }
        
        month = str.upper(date_str[:3])
        year = 2000+int(date_str[-2:])
        if month in de_to_en_month_abbr:
            month=de_to_en_month_abbr[month]
        date_str=f'{month} {year}'
        return pd.to_datetime(datetime.strptime(date_str, '%b %Y'))


    # Entfernen der letzten zwei Zeilen
    daten = daten.iloc[:-2]

    # Entfernen von führenden Leerzeichen in der 'Werk'-Spalte und Filtern von Zeilen, die mit "Total" beginnen
    daten['Werk'] = daten['Werk'].str.strip()
    daten = daten[~daten['Werk'].str.startswith('Total')]

    # Erstellen einer neuen Spalte, die Werte aus 'Werk' übernimmt, wenn sie mit '-' beginnen
    # und den vorherigen Wert beibehält, falls sie nicht mit '-' beginnen, 
    # mit Hilfe der forward fill Methode
    daten['neue Spalte'] = daten['Werk'].apply(lambda x: x if x.startswith('-') else None)
    daten['neue Spalte'].ffill(inplace=True)

    # Entfernen des ersten Zeichens (vermutlich '-') und führender Leerzeichen in 'neue Spalte'
    daten['neue Spalte'] = daten['neue Spalte'].str[1:].str.strip()

    # Aufteilen der 'neue Spalte' in 'material' und 'kurztext' basierend auf dem ersten Leerzeichen
    daten[['material', 'kurztext']] = daten['neue Spalte'].str.split(' ', n=1, expand=True)

    # Verschieben der neuen Spalten an den Anfang des DataFrames
    cols = daten.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    daten = daten[cols]


    # Umschmelzen des DataFrames, um ein langes Format zu erhalten
    daten = daten[~daten['Werk'].str.startswith('-')]
    daten = daten.melt(id_vars=daten.columns[:4], var_name='Month', value_name='Absatz')

    # Extrahieren der letzten 6 Zeichen der 'Month'-Spalte für das Monatsformat
    daten['Month'] = daten['Month'].str[-6:]
    daten[['Werksnummer','Werksname']]=daten['Werk'].str.split(' ', n=1, expand=True)
    daten.drop(['Werk'], axis=1, inplace=True)


    # Zurücksetzen des Indexes, um nach dem Löschen von Zeilen einen sauberen Index zu haben
    daten.reset_index(drop=True, inplace=True)
    daten.drop(['neue Spalte'], axis=1,inplace=True)
    cols = daten.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    daten = daten[cols]

    #Nan mit 0 auffüllen
    daten['Absatz']=daten['Absatz'].fillna(0)
    daten['Month']=daten['Month'].apply(convert_to_datetime_MVER)

    return daten

df= io_df('sds','IO')
df.tail()

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_3088\3688759146.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daten['neue Spalte'].ffill(inplace=True)


,Werksnummer,Werksname,material,kurztext,Month,Absatz
1843,0042,Villeroy&Boch Trading Shanghai,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,2023-12-01,18.0
1844,0058,MONDIAL SA,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,2023-12-01,16.0
1845,0078,Villeroy & Boch Wellness,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,2023-12-01,316.0
1846,0099,Villeroy & Boch Magyarország K,UBQ170OBE2V-01,Oberon bathtub 170x75 rect,2023-12-01,0.0
1847,0054,Villeroy & Boch Gustavsberg AB,FG0603809,Vatette 1219 Blanking-Pc 10,2023-12-01,1190.0


## Gross Demand Tabelle

In [99]:
def gross_demand_df(path:str,sheetname:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Beispielartikel_Bestandsoptimierung_ab2019.xlsx')
    df=pd.read_excel(excel,sheetname)

    def convert_to_datetime_MVER(date_str):
        de_to_en_month_abbr = {
        'JAN': 'Jan',  # Januar
        'FEB': 'Feb',  # Februar
        'MAR': 'Mar',  # März
        'APR': 'Apr',  # April
        'MAI': 'May',  # Mai
        'JUN': 'Jun',  # Juni
        'JUL': 'Jul',  # Juli
        'AUG': 'Aug',  # August
        'SEP': 'Sep',  # September
        'OKT': 'Oct',  # Oktober
        'NOV': 'Nov',  # November
        'DEZ': 'Dec',   # Dezember
        'MRZ': 'Mar'
        }
        
        month = str.upper(date_str[:3])
        year = 2000+int(date_str[-2:])
        if month in de_to_en_month_abbr:
            month=de_to_en_month_abbr[month]
        date_str=f'{month} {year}'
        return pd.to_datetime(datetime.strptime(date_str, '%b %Y'))

    df.rename(columns={'DC (Gross Demand after set & network drilldown) all DC': 'Werk'}, inplace=True)
    df.drop(['Unnamed: 20', 'TOTAL'], axis=1, inplace=True)
    df['Material'] = df[df.drop(['Werk'], axis=1).isnull().all(axis=1)]['Werk']
    df['Material'] = df['Material'].fillna(method='ffill')
    df = df[df['Werk'] != df['Material']]
    # Clear lines where 'Total' appears in the Werk column
    df = df[df['Werk'].str.contains('Total') == False]  
    df = df[df['Werk'].str.contains('TOTAL') == False]
    df.fillna(0, inplace=True)
    df = pd.melt(df, id_vars=['Material', 'Werk'], var_name='Month', value_name='Gross Demand')
    df['Werk'] = df['Werk'].astype(int)
    df['Material'] = df['Material'].str[1:]
    df['Werk']=df['Werk'].apply(werk_str)
    df['Month']=df['Month'].apply(lambda x:x.replace('.',' '))
    df['Month']=df['Month'].apply(convert_to_datetime_MVER)
    return df

df=gross_demand_df('sds','Gross Demand')
df.head()

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_3088\4222488388.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Material'] = df['Material'].fillna(method='ffill')


18

## Bestandstabelle

In [106]:
def bestand_df(path:str) -> pd.DataFrame:
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Bestand 012019 - 022024.xlsx')
    bestand_df=pd.read_excel(excel,0,header=14)
    bedingung_st=bestand_df.iloc[0]=='ST'
    bedingung_eur=bestand_df.iloc[0]=='EUR'
    # Maske für nur die Mengen und Bruttowert
    mengen_ST=bestand_df.loc[:,bedingung_st]
    mengen_ST=mengen_ST.iloc[2:]

    bruttowert_eur=bestand_df.loc[:,bedingung_eur]
    bruttowert_eur=bruttowert_eur.iloc[2:]
    cols=bestand_df.columns.to_list()
    cols[0]='Artikel'
    cols[1]='Kurztext'
    cols[2]='Werk'
    cols[3]='Name des Werks'
    bestand_df.columns=cols
    bestand_df=bestand_df.iloc[2:,:4]
    bestand_menge=pd.concat([bestand_df,mengen_ST],axis= 1)
    #Dataframes in richtige Form bringen
    bestand_menge=bestand_menge.melt(id_vars=bestand_menge.columns[:4],var_name='Month', value_name='Menge in ST')
    bruttowert_eur=bruttowert_eur.melt(var_name='Variable', value_name='Bruttowert in EUR').drop(['Variable'],axis=1)

    final_df= pd.concat([bestand_menge,bruttowert_eur],axis=1)
    final_df=final_df.fillna({"Menge in ST": 0, "Bruttowert in EUR": 0})
    final_df['Month']=final_df['Month'].apply(lambda x: pd.to_datetime(x, format='%m.%Y'))
    return final_df

df=bestand_df('sds')
df.head()

C:\Users\DjauschanFedaie\AppData\Local\Temp\ipykernel_3088\3686525987.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df=final_df.fillna({"Menge in ST": 0, "Bruttowert in EUR": 0})


,Artikel,Kurztext,Werk,Name des Werks,Month,Menge in ST,Bruttowert in EUR
0,560050R1P,Subway 2.0 WWC for V,0011,VILLEROY & BOCH AG,2019-01-01,0,0.00
1,560050R1P,Subway 2.0 WWC for V,0028,Villeroy&Boch Trading Shenzhen,2019-01-01,0,0.00
2,560050R1P,Subway 2.0 WWC for V,0041,Villeroy & Boch OOO,2019-01-01,0,0.00
3,560050R1P,Subway 2.0 WWC for V,0042,Villeroy&Boch Trading Shanghai,2019-01-01,1774,178802.06
4,61630001,Loop & Friends Unter,0011,VILLEROY & BOCH AG,2019-01-01,2733,140257.56


## Lagerkosten je DC Tabelle

In [109]:
def lagerkosten_je_standort():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lagerkosten je DC.xlsx')
    lagerkosten_df=pd.read_excel(excel,'Lagerkosten je Standort', header=1)
    lagerkosten_df=lagerkosten_df.iloc[0:8,1:]
    lagerkosten_df=lagerkosten_df.rename(columns={"Unnamed: 1": "Standort"})
    lagerkosten_df['DC']=lagerkosten_df['DC'].apply(int)
    lagerkosten_df['DC']=lagerkosten_df['DC'].apply(werk_str)
    return lagerkosten_df

df= lagerkosten_je_standort()
df['DC']

0    0011
1    0067
2    0099
3    0058
4    0044
5    0049
6    0042
7    0028
Name: DC, dtype: object

## Menge pro Palette

In [5]:
def menge_pro_palette():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lagerkosten je DC.xlsx')
    pallete_df=pd.read_excel(excel,'Menge pro Palette')
    pallete_df=pallete_df.iloc[:-1]
    return pallete_df

df=menge_pro_palette()
df.head()

,Article,Quantity per Pallet (DIN-Format)
0,GB1921900201,99999
1,GB1921100240,90000
2,GB1921100005,80000
3,TVP00000100400,64000
4,TVP00000521000,64000


## Lieferzeiten und Lieferwege DC-Netzwerk

In [113]:
def lieferwege():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lieferzeiten und Lieferwege DC-Netzwerk.xlsx')
    lieferwege_df=pd.read_excel(excel,'interne Lieferwege')
    return lieferwege_df

lieferwege().head()

,Source DC,DC,Transshipment DC 1,Transshipment DC 2,Transshipment DC 3,Unnamed: 5
0,5,0001,NaN,NaN,NaN,
1,5,0005,NaN,NaN,NaN,
2,5,0007,NaN,NaN,NaN,
3,5,0008,NaN,NaN,NaN,
4,5,0009,NaN,NaN,NaN,


In [146]:
def werk_str(x):
    x=str(x)
    length_werk=len(x)
    diff=4-length_werk
    final_werk= diff*'0'+x
    return final_werk

def lieferzeiten():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lieferzeiten und Lieferwege DC-Netzwerk.xlsx')
    lieferzeiten_df=pd.read_excel(excel,'Lieferzeiten_gefiltert',dtype=str)
    lieferzeiten_df=lieferzeiten_df.iloc[:,:3]
    cols=lieferzeiten_df.columns.to_list()
    cols[2]="Lieferzeiten in d"
    cols=[str.strip(x) for x in cols]
    lieferzeiten_df.columns=cols
    return lieferzeiten_df

lieferzeiten().head()

,Source DC,DC,Lieferzeiten in d
0,0011,0012,60
1,0011,0025,60
2,0011,0028,60
3,0011,0041,22
4,0011,0042,60


## Standarskosten &-lieferzeit

In [133]:
def standardkosten():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Standardkosten &-lieferzeit.xlsx')
    standardkosten_df=pd.read_excel(excel,0)
    return standardkosten_df

standardkosten().head()

,Artikel,Standardkosten (€),Standardbereitstellungszeit (Arbeitstage)
0,560050R1P,109.39,1
1,5614R001P,92.11,1
2,61630001,63.28,1
3,92218068,13.64,1
4,92219900,7.98,1


## Verfügbare Menge pro DC

In [160]:
import re
def menge_pro_dc():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')
    file=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Verfügbare Menge pro DC.xlsx')


    def data_man(werk:str,number_of_sheet:int):
        menge_dc_df=pd.read_excel(excel,number_of_sheet)
        #alle Spalten führende Leerzeichen wegmachen
        cols=menge_dc_df.columns.to_list()
        cols=[str.strip(x) for x in cols]
        menge_dc_df.columns=cols
        #alle unnamed Spalten raus
        menge_dc_df = menge_dc_df.loc[:, ~menge_dc_df.columns.str.contains('^Unnamed')]
        menge_dc_df.drop(['Summe:'], axis=1,inplace= True)
        menge_dc_df=menge_dc_df.iloc[:-2]
        menge_dc_df=menge_dc_df.melt(id_vars=menge_dc_df.columns[0],var_name='Material',value_name='Menge')
        menge_dc_df[['Material','Kurztext']]=menge_dc_df['Material'].str.split(" ",n=1,expand=True)
        cols=menge_dc_df.columns.to_list()
        #Reihenfolge der Spalten angepasst
        menge_dc_df=menge_dc_df[cols[0:2]+cols[-1:]+cols[2:3]]
        menge_dc_df['Werk']=werk
        
        return menge_dc_df


    final_df=pd.DataFrame()
    for i in range(0,len(file.sheet_names)):
        data=data_man(file.sheet_names[i],i)
        final_df=pd.concat([final_df,data],axis=0)
    final_df.reset_index(drop=True,inplace=True)
    final_df['Monat'] = final_df['Monat'].apply(lambda x: pd.to_datetime(x.strftime('%Y-%m-01')))
    pattern = re.compile(r'\b\d{4}\b')
    def extract_number(s):
        match = pattern.search(s)
        return match.group() if match else None
    final_df['Werk']=final_df['Werk'].apply(extract_number)

    return final_df

a=menge_pro_dc()
a.head()




,Monat,Material,Kurztext,Menge,Werk
0,2019-01-01,5614R001P,Subway 2.0 WWC DirectFlush verpackt,0.0,0005
1,2019-02-01,5614R001P,Subway 2.0 WWC DirectFlush verpackt,0.0,0005
2,2019-03-01,5614R001P,Subway 2.0 WWC DirectFlush verpackt,0.0,0005
3,2019-04-01,5614R001P,Subway 2.0 WWC DirectFlush verpackt,0.0,0005
4,2019-05-01,5614R001P,Subway 2.0 WWC DirectFlush verpackt,0.0,0005


In [3]:
def lieferwege():
    excel=pd.ExcelFile('c:\\celver\\scm_optimization\\v_b\\raw_data\\Lieferzeiten und Lieferwege DC-Netzwerk.xlsx')
    lieferwege_df=pd.read_excel(excel,'interne Lieferwege',dtype=str)
    return lieferwege_df



lieferwege().head()

,Source DC,DC,Transshipment DC 1,Transshipment DC 2,Transshipment DC 3,Unnamed: 5
0,0005,0001,NaN,NaN,NaN,
1,0005,0005,NaN,NaN,NaN,
2,0005,0007,NaN,NaN,NaN,
3,0005,0008,NaN,NaN,NaN,
4,0005,0009,NaN,NaN,NaN,
